In [12]:
import pandas as pd
df = pd.read_csv("C:\\Users\\VICTUS\\Documents\\GitHub\\ppcc_dgfuchile\\eventos\\2024-04-30.csv")
df.drop(['pp', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df.head()
df.index.name = 'index'
df.to_csv("C:\\Users\\VICTUS\\Documents\\GitHub\\ppcc_dgfuchile\\usuarios\\usuarios.csv")


,nombre,mail,comuna,lat,lon,alias,grupo
0,Vicente López,vicentelopez.v@gmail.com,Peñaflor,-33.606575,-70.906942,Huaso,MET-01
1,Maria Fernanda Ruedlinger,maria.ruedlinger@ug.uchile.cl,Maipu,-33.491232,-70.766251,Saturno,MET-01
2,María Victoria Atisha,maria.atisha@ug.uchile.cl,Ñuñoa,-33.460082,-70.592846,Botitas,MET-01
3,Diego Ignacio Faúndez,dfaundezortega@gmail.com,La Florida,-33.534556,-70.550194,Fortnite,MET-01
4,Matías Valentín Guzmán,matias.guzmanparra@gmail.com,La Florida,-33.531720,-70.556389,KIKE,MET-01


In [8]:
import pandas as pd
df = pd.read_csv("C:\\Users\\VICTUS\\Documents\\GitHub\\ppcc_dgfuchile\\eventos\\2024-04-24.csv")
df.loc[df.mail == "vicentelopez.v@gmail.com", 'pp'] = 10

In [9]:
df

,index,nombre,mail,comuna,lat,lon,alias,grupo,pp
0,0,Vicente López,vicentelopez.v@gmail.com,Peñaflor,-33.606575,-70.906942,Huaso,MET-01,10.0
1,1,Maria Fernanda Ruedlinger,maria.ruedlinger@ug.uchile.cl,Maipu,-33.491232,-70.766251,Saturno,MET-01,NaN
2,2,María Victoria Atisha,maria.atisha@ug.uchile.cl,Ñuñoa,-33.460082,-70.592846,Botitas,MET-01,NaN
3,3,Diego Ignacio Faúndez,dfaundezortega@gmail.com,La Florida,-33.534556,-70.550194,Fortnite,MET-01,NaN
4,4,Matías Valentín Guzmán,matias.guzmanparra@gmail.com,La Florida,-33.531720,-70.556389,KIKE,MET-01,NaN
...,...,...,...,...,...,...,...,...,...
143,143,William Fevre,x,y,-33.671400,-70.581700,VisMet,vismet,NaN
144,144,Mapocho Los Almendros,x,y,-33.370300,-70.450800,VisMet,vismet,NaN
145,145,San Miguel,x,y,-33.506700,-70.650800,VisMet,vismet,NaN
146,146,Río Molina antes Junta San Francisco,x,y,-33.373300,-70.396100,VisMet,vismet,NaN
